In [1]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, f1_score # Using F1 score

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks
import all_metrics as all_metrics # Assuming your custom metrics function is available
import credit_card_fraud_utils as ccf


In [2]:
config = ccf.Config()
# Load the dataset
df = ccf.load_data(config)

# Display basic information
print(f"Dataset Shape: {df.shape}")
print(f"Number of Transactions: {df.shape[0]}")
print(f"Number of Features: {df.shape[1]}")

# Display the first few rows
df.head()

Dataset Shape: (284807, 31)
Number of Transactions: 284807
Number of Features: 31


Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  
1  0.125895 -0.008983  0.014724    2.69      0  
2 -0.139097 -0.055353 -0.059752  378.66      0  
3 -0.221929  0.062723  0.061458  123.50      0  
4  0.502292  0.219422  0.215153   69.99      0  

[5 rows x 31 columns]

In [3]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Class'])
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [4]:
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()), # Keep scaling
    ('resampler', None),          # Placeholder
    ('classifier', RandomForestClassifier(
        random_state=42,
        n_jobs=-1 # Use all cores for RF too
    ))
])

In [5]:

rf_param_ranges = {
    'classifier__n_estimators': [100, 200, 300, 400, 500], # Number of trees
    'classifier__max_depth': [10, 20, 30, None], # Deeper trees often okay for RF; None means no limit until leaves are pure or min_samples hit
    'classifier__max_features': ['sqrt', 'log2', 0.3], # Number of features per split ('sqrt' is common)
    'classifier__min_samples_split': [2, 5, 10], # Min samples to split node
    'classifier__min_samples_leaf': [1, 3, 5],    # Min samples per leaf
    'classifier__bootstrap': [True] # Usually True for RF
    # 'classifier__criterion': ['gini', 'entropy'] # Can also test split criterion
}

# Resampler parameters (same as before)
smote_k_neighbors = [3, 5, 10]
adasyn_n_neighbors = [3, 5, 10]
rus_sampling_strategy = [0.2, 0.35, 0.5]
ros_sampling_strategy = [0.5, 0.7, 1.0]

param_grid_rf = []

# --- Configuration for NO Resampling (Test RF's internal weighting) ---
param_grid_rf.append({
    'resampler': [None],
    **rf_param_ranges,
    'classifier__class_weight': [None, 'balanced', 'balanced_subsample'] # RF's way to handle imbalance
})


# --- Configurations for Resampling Methods --
resampling_options_list = [
    {'resampler': [SMOTE(random_state=42)], 'resampler__k_neighbors': smote_k_neighbors},
    {'resampler': [ADASYN(random_state=42)], 'resampler__n_neighbors': adasyn_n_neighbors},
    {'resampler': [RandomOverSampler(random_state=42)], 'resampler__sampling_strategy': ros_sampling_strategy},
    {'resampler': [RandomUnderSampler(random_state=42)], 'resampler__sampling_strategy': rus_sampling_strategy},
    {'resampler': [TomekLinks(sampling_strategy='auto')]},
    {'resampler': [NearMiss(version=1)]}
]

# --- Configurations for Resampling Methods (Turn off internal weighting) ---

for resampler_config in resampling_options_list:
    config = {
        **resampler_config,
        **rf_param_ranges,
        'classifier__class_weight': [None] # Use None when external resampling is active
    }
    param_grid_rf.append(config)

In [8]:
n_iterations_rf = 75 # Or adjust as needed
scoring_metric = 'f1'
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

random_search_rf = RandomizedSearchCV(
    pipeline_rf,
    param_distributions=param_grid_rf,
    n_iter=n_iterations_rf,
    scoring=scoring_metric,
    refit=scoring_metric,
    cv=cv_strategy,
    verbose=1, # Show progress
    n_jobs=-1, # Use all cores
    random_state=42
)

print(f"\nStarting RandomizedSearchCV for RandomForest with n_iter={n_iterations_rf}...")
random_search_rf.fit(X_train, y_train) # Uncomment to run


Starting RandomizedSearchCV for RandomForest with n_iter=75...
Fitting 5 folds for each of 75 candidates, totalling 375 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('resampler', None),
                                             ('classifier',
                                              RandomForestClassifier(n_jobs=-1,
                                                                     random_state=42))]),
                   n_iter=75, n_jobs=-1,
                   param_distributions=[{'classifier__bootstrap': [True],
                                         'classifier__class_weight': [None,
                                                                      'balanced',
                                                                      'balanced_subs...
                                        {'classifier__bootstrap': [True],
                                         'classifier__class_weight': [None],
                                         'classifier__max_depth': [10, 20, 30,
                                                                   None],
                                         'classifier__max_features': ['sqrt',
                                                                      'log2',
                                                                      0.3],
                                         'classifier__min_samples_leaf': [1, 3,
                                                                          5],
                                         'classifier__min_samples_split': [2, 5,
                                                                           10],
                                         'classifier__n_estimators': [100, 200,
                                                                      300, 400,
                                                                      500],
                                         'resampler': [NearMiss()]}],
                   random_state=42, refit='f1', scoring='f1', verbose=1)

In [9]:

print("\nBest Random Forest Parameters Found:")
print(random_search_rf.best_params_)
print("\nBest CV " + scoring_metric.upper() + " Score:")
print(random_search_rf.best_score_)


Best Random Forest Parameters Found:
{'resampler__sampling_strategy': 0.7, 'resampler': RandomOverSampler(random_state=42), 'classifier__n_estimators': 200, 'classifier__min_samples_split': 5, 'classifier__min_samples_leaf': 3, 'classifier__max_features': 'sqrt', 'classifier__max_depth': 30, 'classifier__class_weight': None, 'classifier__bootstrap': True}

Best CV F1 Score:
0.8556325149342255


In [7]:

best_pipeline_rf = random_search_rf.best_estimator_
results_rf = all_metrics.calculate_model_metrics(
    best_pipeline_rf, X_test, y_test, 'Random Forest (RandomizedSearch Best)'
)



--- Evaluating Best XGBoost Model on Test Set ---


AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [ ]:

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical





# --- Define Focused Bayes Search Space (EXAMPLE - adapt based on YOUR RF RandomizedSearch results) ---

# search_spaces_rf_bayes = {
#     'resampler': Categorical([None]),
#     'classifier__class_weight': Categorical(['balanced']),
#     'classifier__n_estimators': Integer(200, 500),
#     'classifier__max_depth': Integer(15, 40), # Explore around the likely deep best value
#     'classifier__max_features': Categorical(['sqrt', 'log2']), # Or Real(0.2, 0.7)
#     'classifier__min_samples_split': Integer(2, 8),
#     'classifier__min_samples_leaf': Integer(1, 5),
#     'classifier__bootstrap': Categorical([True])
# }

# --- Configure BayesSearchCV ---
# n_iterations_rf_bayes = 50 # Or adjust
# scoring_metric = 'f1'
# cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# bayes_search_rf = BayesSearchCV(
#     estimator=pipeline_rf,
#     search_spaces=search_spaces_rf_bayes,
#     n_iter=n_iterations_rf_bayes,
#     scoring=scoring_metric,
#     cv=cv_strategy,
#     n_jobs=-1,
#     refit=True,
#     random_state=42,
#     verbose=1
# )

# --- Run Search ---
# print(f"\nStarting BayesSearchCV for RandomForest with n_iter={n_iterations_rf_bayes}...")
# bayes_search_rf.fit(X_train, y_train) # Uncomment

# --- Evaluate ---
# print("\nBest RandomForest Parameters Found (BayesSearchCV):")
# print(bayes_search_rf.best_params_)
# print("\nBest CV " + scoring_metric.upper() + " Score (BayesSearchCV):")
# print(bayes_search_rf.best_score_)
# Evaluate bayes_search_rf.best_estimator_ on test set...